In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 10
epochs = 100
#data_augmentation = True
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-4),padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3),kernel_regularizer=regularizers.l2(1e-4),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(1e-4),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(1e-4),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(1e-4),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(1e-4),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001,momentum=0.9,decay=0.000001),
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)

steps = int(x_train.shape[0] / 64)

results=model.fit_generator(it_train,steps_per_epoch=steps,epochs=epochs,validation_data=(x_test, y_test),shuffle=True)




In [0]:
from matplotlib import pyplot


scores = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


def summarize_diagnostics(results):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(results.history['loss'], color='blue', label='train')
	pyplot.plot(results.history['val_loss'], color='orange', label='test')

 
summarize_diagnostics(results)